# User Inputs  

## Ask the user for the loan parameters

In [3]:
amount = float(input("How much money do you need to borrow?: "))
print()
monthly_term = int(input("In how many months do you want to finish paying your loan?: "))
print()
interest_rate = float(input("What's the annual interest rate they are charging you?\nPlease enter your answer without the '%' symbol: "))/100
print()
amortization_type = int(input("How would you like to pay each month?\n1. I want to pay principal and interest each month.\n2. I want to pay only interests each month.\n3. I want to pay principal and interest at the end of the loan term.\n Please input the option number (1, 2 or 3): "))
print()

# Classes  

## Create a class for loans

In [13]:
class Loan:
    def __init__(self, input_amount, input_monthly_term, input_interest_rate, input_amortization_type=1):
        self.input_amount = amount
        self.input_monthly_term = monthly_term
        self.input_interest_rate = interest_rate
        self.input_amortization_type = amortization_type
    
    # Give a description of the loan main parameters
    def __repr__(self):
        description = "Your loan of {loan_amount} dollars will be paid in {term} months and ".format(loan_amount=amount, term=monthly_term)
        if amortization_type == 1:
            description += "in each month you will have to pay interest and principal."
        elif amortization_type == 2:
            description += "in each month you will have to pay the interest.\nThe principal will be paid in the last month of the term."
        elif amortization_type == 3:
            description += "the interest and principal will be paid at maturity."
        return description

In [14]:
loan1 = Loan(amount, monthly_term, interest_rate, amortization_type)
print(loan1)

Your loan of 50000.0 dollars will be paid in 12 months and in each month you will have to pay interest and principal.
